# Multiple extended gcd algorithm using LLL

## Date : 2021.04.21
## Copyright by HyewonSung

In [71]:
import sys
import math
import numpy as np
from numpy import linalg as la

from fractions import Fraction
from typing import Sequence
from typing import List
from functools import reduce


### input vector w

In [100]:
w = np.array([13, 143, 156, 1300, 39, 26, 117, 1170])
alpha = 3/4 #LLL alpha

print("input vector w : ", w)


input vector w :  [  13  143  156 1300   39   26  117 1170]


### Defining Vector Class, GSO process, LLL reduction 

In [101]:
#Define class vector
class Vector(list):
            
    def sdot(self) :  
       
        return self.dot(self)
    
    def dot(self, rhs: "Vector") :
        rhs = Vector(rhs)
        assert len(self) == len(rhs)
        return sum(map(lambda x: x[0] * x[1], zip(self, rhs)))
    
    def proj_coff(self, rhs: "Vector") :
        rhs = Vector(rhs)
        assert len(self) == len(rhs)
        return self.dot(rhs) / self.sdot()
    
    def proj(self, rhs: "Vector") -> "Vector":
        rhs = Vector(rhs)
        assert len(self) == len(rhs)
        return self.proj_coff(rhs) * self
    
    def __sub__(self, rhs: "Vector") -> "Vector":
        rhs = Vector(rhs)
        assert len(self) == len(rhs)
        return Vector(x - y for x, y in zip(self, rhs))
    
    def __mul__(self, rhs) -> "Vector":
        return Vector(x * rhs for x in self)
    
    def __rmul__(self, lhs) -> "Vector":
        return Vector(x * lhs for x in self)
    
    def __repr__(self) -> str:
        return "[{}]".format(", ".join(str(x) for x in self))

# GSO
def gramschmidt(v: Sequence[Vector]) -> Sequence[Vector]:
    u: List[Vector] = []
    for vi in v:
        ui = Vector(vi)
        for uj in u:
            ui = ui - uj.proj(vi)
        if any(ui):
            u.append(ui)
    return u


def dot(a,b):
    a=np.array(a)
    b=np.array(b)
    u=a*b
    C=u[0]
    i=1
    while i<len(a):
        C=C+u[i]
        i=i+1
    return C

def mult(v,B):
    B=np.matrix(B)
    v=np.array(v)
    C=v*(B.T)
    return sum(map(lambda x: x , C))

# LLL
def LLLreduction(basis: Sequence[Sequence[int]], delta: float):
    
    n = len(basis)
    basis = list(map(Vector, basis))
    ortho = gramschmidt(basis)
    def mu(i: int, j: int) :
        return ortho[j].proj_coff(basis[i])
    k = 1
    while k < n:
        for j in range(k - 1, -1, -1):
            mu_kj = mu(k, j)
            if abs(mu_kj) > 0.5:
                basis[k] = basis[k] - basis[j] * round(mu_kj)
                ortho = gramschmidt(basis)
        if ortho[k].dot(ortho[k])>= (delta-((mu(k, k-1))**2))*(ortho[k-1].dot(ortho[k-1])):
            k += 1
            #print("Lovasz holds")
        else:
            #print("Lovasz fails")
            basis[k], basis[k - 1] = basis[k - 1], basis[k]
            ortho = gramschmidt(basis)
            k = max(k - 1, 1)
    return basis

### Setting integer r so thar r>2^((n-1)/2) * length of w

In [102]:
#Set an integer r 

n = len(w)
wnorm = np.linalg.norm(w)
r = math.ceil((2**((n-1)/2)*wnorm))

#print(wnorm)
#print(r)


1766.2785737249942
19984


### Setting a new basis matrix B_nx(n+1) = [I_nxn | r*w]

In [109]:
#Set a basis matrix B_nx(n+1)

identity = np.eye(n)
rw = r*w

bmatrix = np.column_stack((identity, rw))
print("Basis matrix : \n",bmatrix)

Basis matrix : 
 [[1.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00 2.597920e+05]
 [0.000000e+00 1.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00 2.857712e+06]
 [0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00 3.117504e+06]
 [0.000000e+00 0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00 2.597920e+07]
 [0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 1.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00 7.793760e+05]
 [0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  1.000000e+00 0.000000e+00 0.000000e+00 5.195840e+05]
 [0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 1.000000e+00 0.000000e+00 2.338128e+06]
 [0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 1.000000e+00 2.338128e+07]]


### Compute X matrix by using LLL algorithm. (i.e X = LLL(B))

In [104]:
# Running LLL algorithm with B matrix

X=np.array(LLLreduction(bmatrix, 0.75))
print("LLL reduced basis : \n", X)

LLL reduced basis : 
 [[-1.00000e+00 -1.00000e+00  1.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00]
 [ 1.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00 -1.00000e+00
   1.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00]
 [ 0.00000e+00 -1.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   1.00000e+00  1.00000e+00  0.00000e+00  0.00000e+00]
 [-1.00000e+00  1.00000e+00 -1.00000e+00  0.00000e+00  0.00000e+00
   1.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00]
 [ 0.00000e+00  0.00000e+00  1.00000e+00 -1.00000e+00  0.00000e+00
  -1.00000e+00  0.00000e+00  1.00000e+00  0.00000e+00]
 [-1.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00 -2.00000e+00
  -1.00000e+00  1.00000e+00  0.00000e+00  0.00000e+00]
 [ 0.00000e+00 -3.00000e+00 -2.00000e+00 -1.00000e+00 -1.00000e+00
  -1.00000e+00 -2.00000e+00  2.00000e+00  0.00000e+00]
 [ 1.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00  0.00000e+00  2.59792e+05]]


### Output d (gcd value of elements of vector w) and x vector 

In [105]:
# main

#Output d = gcd(d1, ..., dn) where w = (d1, ..., dn)
d = X[n-1][n] / r
print("gcd of d1, ..., dn (d_i are elemts of input w vector) : ",d)

#Output (x1, ..., xn) in Z^n such that x1d1 + x2d2 + ... + xndn = d
outputx = X[n-1][:n]
print("outputx vector : ", outputx)

#check output d with x
print("inner product with w and outputx : ", dot(w,outputx))

gcd of d1, ..., dn (d_i are elemts of input w vector) :  13.0
outputx vector :  [1. 0. 0. 0. 0. 0. 0. 0.]
inner product with w and outputx :  13.0
